### API DOCUMENTATION:

https://docs.edenai.co/reference/ocr_financial_parser_create

In [3]:
import json
import requests
from config import Config
import os

image_path = os.path.join(os.getcwd(), "sample_receipts/QFC_1.jpg")

headers = {"Authorization": f"Bearer {Config.EDENAI_API_KEY}"}

url = "https://api.edenai.run/v2/ocr/financial_parser"
data = {
    "providers": "amazon", # DO NOT CHANGE PROVIDER
    "language": "en",
    "document_type" : "receipt",
}
files = {'file': open(image_path, 'rb')}

response = requests.post(url, data=data, files=files, headers=headers)
result = response.json()  # Directly get the JSON response
print(json.dumps(result, indent=4))

{
    "amazon": {
        "status": "success",
        "extracted_data": [
            {
                "customer_information": {
                    "name": "jobs.qfc.com",
                    "id_reference": null,
                    "mailling_address": null,
                    "billing_address": null,
                    "shipping_address": null,
                    "service_address": null,
                    "remittance_address": "2746 NE 45th Street",
                    "email": null,
                    "phone": null,
                    "vat_number": null,
                    "abn_number": null,
                    "gst_number": null,
                    "pan_number": null,
                    "business_number": null,
                    "siret_number": null,
                    "siren_number": null,
                    "customer_number": null,
                    "coc_number": null,
                    "fiscal_number": null,
                    "registration_number": null,
